# Aplicação do Algoritmo A* para planejamento de visita à pontos turísticos na Itália

## Introdução:

### Motivação:

Este projeto tem como objetivo apresentar as características e o funcionameto do algoritmo de busca A* (A-Estrela).

### Seleção do problema e estados objetivos:

Domínio do problema: Proposição de pacote de viagem de São Paulo aos 3 pontos turísticos mais bem avaliados da Itália com o deslocamento menos custoso.

De acordo com o TripAdvisor (plataforma digital reconhecida como o maior site de viagens do mundo), os pontos turísticos mais bem avaliados são:
- Sassi di Matera (Província de Taranto)
- St Peter's Basílica (Roma)
- Gardaland Park (Verona)

<img src="https://guiati9.dev/img/posts_img/tripadvisor.png"/>

[Trip Advisor - Italy Vacation](https://www.tripadvisor.com/Tourism-g187768-Italy-Vacations.html)[5]

Sendo assim, a solução proposta deve apresentar o caminho menos custoso à ser percorrido para um turista que pretenda visitar estes três pontos.

Considerando o contexto de agentes de busca, é possível dizer que os três destinos podem ser considerados os estados objetivos do problema, desde que ao fim da execução do algoritmo, todos tenham sido visitados. Enquanto o ponto de partida será São Paulo.

## Metodologia:


### Estrutura de dados:

A estrutura de dados desse projeto é composta por um grafo, uma tabela de distâncias em linha reta entre as estações, uma tabela de preço médio de hoteis da região de cada estação e uma tabela de heurísticas que é resultado do processamento das informações apresentadas pelas duas tabelas anteriores.

In [1]:
# This class represent a graph
class Graph:

    # Initialize the class
    def __init__(self, graph_dict=None, directed=True):
        self.graph_dict = graph_dict or {}
        self.directed = directed
        if not directed:
            self.make_undirected()

    # Create an undirected graph by adding symmetric edges
    def make_undirected(self):
        for a in list(self.graph_dict.keys()):
            for (b, dist) in self.graph_dict[a].items():
                self.graph_dict.setdefault(b, {})[a] = dist

    # Add a link from A and B of given distance, and also add the inverse link if the graph is undirected
    def connect(self, A, B, distance=1):
        self.graph_dict.setdefault(A, {})[B] = distance
        if not self.directed:
            self.graph_dict.setdefault(B, {})[A] = distance

    # Get neighbors or a neighbor
    def get(self, a, b=None):
        links = self.graph_dict.setdefault(a, {})
        if b is None:
            return links
        else:
            return links.get(b)

    # Return a list of nodes in the graph
    def nodes(self):
        s1 = set([k for k in self.graph_dict.keys()])
        s2 = set([k2 for v in self.graph_dict.values() for k2, v2 in v.items()])
        nodes = s1.union(s2)
        return list(nodes)

In [2]:
# This class represent a node
class Node:

    # Initialize the class
    def __init__(self, name:str, parent:str):
        self.name = name
        self.parent = parent
        self.g = 0 # Distance to start node
        self.h = 0 # Distance to goal node
        self.f = 0 # Total cost

    # Compare nodes
    def __eq__(self, other):
        return self.name == other.name

    # Sort nodes
    def __lt__(self, other):
         return self.f < other.f

Este trecho é uma adaptação do código cedido por [annytab.com](https://www.annytab.com/a-star-search-algorithm-in-python/)[1] ao domínio do problema.

#### Disposição das estações e o custo aproximado das rotas de trem na Itália

Custo de rotas           |  Principais estações| SP - Itália
:-------------------------:|:-------------------------:|:-------------------------:
<img src="https://guiati9.dev/img/posts_img/cost-map-italy.png" width="400px"/>  |  <img src="https://guiati9.dev/img/posts_img/italy-train-map.png" width="400px"/> | <img src="https://guiati9.dev/img/posts_img/gflights.png" width="400px"/>
[Rick Steves - Travel Tips](https://www.ricksteves.com/travel-tips/transportation/trains/italy-rail-passes)[2]|[Maps Italy](https://maps-italy.com/map-of-train-routes-in-italy)[3]|[Google Flight Tracker[4]](https://www.google.com/travel/explore?g2lb=2502548%2C4258168%2C4260007%2C4270442%2C4274032%2C4291318%2C4305595%2C4306835%2C4308226%2C4317915%2C4326765%2C4328159%2C4329288%2C4366684%2C4373848%2C4381263%2C4382624%2C4385383%2C4386665%2C4395397%2C4270859%2C4284970%2C4291517%2C4316256%2C4356900&hl=en&gl=br&un=1&tfs=CBkQAxopEgoyMDIwLTA2LTMwag0IAhIJL20vMDIycGZtcgwIBBIIL20vMDNyamoaKRIKMjAyMC0wNy0wNGoMCAQSCC9tLzAzcmpqcg0IAhIJL20vMDIycGZtQAFIAVIDQlJMcAE&curr=BRL&authuser=0&origin=https%3A%2F%2Fwww.google.com&gsas=1)



#### Grafo gerado a partir do custo aproximado, mapa de estações e vôos disponíveis

<img src="https://raw.githubusercontent.com/gatihe/guiati9.dev/master/img/posts_img/italy_routes_prices_4.png" width="1000px"/>

In [3]:
# Create a graph
graph = Graph()
graph.connect('SP', 'Rome', 5560) 
graph.connect('SP', 'Bologna', 6310) 
graph.connect('SP', 'Milano', 5688) 
graph.connect('SP', 'Venice', 4658)
graph.connect('Reggio di Calabria', 'Taranto', 149)
graph.connect('Reggio di Calabria', 'Naples', 323)
graph.connect('Lecce', 'Brindisi', 25)
graph.connect('Brindisi', 'Bari', 99)
graph.connect('Bari', 'Taranto', 49)
graph.connect('Bari', 'Foggia', 124)
graph.connect('Taranto', 'Naples', 149)
graph.connect('Naples', 'Foggia', 248)
graph.connect('Foggia', 'Pescara', 124)
graph.connect('Naples', 'Rome', 248)
graph.connect('Rome', 'Pescara', 74)
graph.connect('Pescara', 'Ancona', 74)
graph.connect('Rome', 'Ancona', 124)
graph.connect('Rome', 'Florence', 248)
graph.connect('Rome', 'Pisa', 248)
graph.connect('Pisa', 'Florence', 49)
graph.connect('Florence', 'Bologna', 149)
graph.connect('Bologna', 'Ancona', 174)
graph.connect('Pisa', 'Genoa', 149)
graph.connect('Genoa', 'Bologna', 124)
graph.connect('Genoa', 'Torino', 124)
graph.connect('Genoa', 'Milano', 124)
graph.connect('Milano', 'Bologna', 223)
graph.connect('Bologna', 'Verona', 99)
graph.connect('Bologna', 'Venice', 174)
graph.connect('Venice', 'Trieste', 74)
graph.connect('Venice', 'Verona', 124)
graph.connect('Verona', 'Milano', 124)
graph.connect('Milano', 'Torino', 174)
graph.connect('Verona', 'Bolzano', 124)

# Make graph undirected, create symmetric connections
graph.make_undirected()

In [4]:
#importanto planilhas para alimentar função heurística
import pandas as pd
import numpy as np
dist_linha_reta = pd.read_csv("distancias_estacoes_italia.csv", index_col = 0)
precos_hotel = pd.read_csv("precos_hotel.csv")

precos_hotel = precos_hotel
tabela_heuristicas = dist_linha_reta

In [5]:
precos_hotel

,Unnamed: 0,Bolzano,Verona,Milano,Torino,Trieste,Venice,Bologna,Genoa,Pisa,...,Pescara,Naples,Foggia,Bari,Brindisi,Lecce,Taranto,Reggio di Calabria,SP,Unnamed: 22
0,NaN,326,75,84,155,170,145,178,178,117,...,238,95,223,106,237,167,145,150,60,NaN


In [6]:
tabela_heuristicas

,Bolzano,Verona,Milano,Torino,Trieste,Venice,Bologna,Genoa,Pisa,Florence,...,Rome,Pescara,Naples,Foggia,Bari,Brindisi,Lecce,Taranto,Reggio di Calabria,SP
Bolzano,0,124,209,325,207,139,223,297,315,305,...,518,501,668,651,744,832,878,821,995,9701
Verona,124,0,138,263,218,98,107,199,197,185,...,411,421,576,575,676,778,813,748,901,9606
Milano,209,138,0,131,356,243,200,120,218,249,...,480,524,659,677,786,892,925,857,975,9502
Torino,325,263,131,0,480,365,296,125,264,318,...,525,600,713,753,865,971,1004,931,1022,9382
Trieste,207,218,356,480,0,114,228,404,342,288,...,429,358,535,487,563,653,690,640,853,9790
Venice,139,98,243,365,114,0,130,290,246,204,...,393,366,534,513,605,701,739,680,861,9685
Bologna,223,107,200,296,228,130,0,192,116,84,...,304,325,471,480,587,690,725,657,797,9564
Genoa,297,199,120,125,404,290,192,0,141,199,...,403,480,592,631,744,849,884,810,900,9411
Pisa,315,197,218,264,342,246,116,141,0,69,...,264,340,450,489,604,709,742,668,764,9451
Florence,305,185,249,318,288,204,84,199,69,0,...,232,281,409,435,546,652,686,613,729,9503


### Função heurística adotada:

#### h(n) = d * p / 100

A fórmula heurística escolhida define h(n) como o produto da distância da estação em linha reta e o preço médio de hotéis da região da estação dividido por 100.

A divisão por 100 é feita para reduzir, de forma proporcional à todas as estações, o valor da heurística com a finalidade de evitar que esta superestime o valor do custo de chegar até o nó (g(n)), ou seja, para que não se torne uma heurística não admissível.

In [7]:
stations_list = list(tabela_heuristicas.columns.values)


tabela_heuristicas.T


def set_heuristics(station_name):
    station_index = stations_list.index(station_name)
    heuristics = [value for value in heuristics_dict.values()][station_index]
    return heuristics

#Aplicando heurística (distância em linha reta ao destino * preco médio de hotel na região/100)
dict_hoteis = precos_hotel.to_dict()
for station in stations_list:
    teste = tabela_heuristicas[station].to_dict()
    #print(station)
    #print(teste)
    for key in dict_hoteis:
        teste[key] = teste.get(key, 1) * dict_hoteis[station][0]/100
    for key in teste:
        tabela_heuristicas[station][key] = teste[key]
heuristics_dict = tabela_heuristicas.to_dict()

tabela_heuristicas


,Bolzano,Verona,Milano,Torino,Trieste,Venice,Bologna,Genoa,Pisa,Florence,...,Rome,Pescara,Naples,Foggia,Bari,Brindisi,Lecce,Taranto,Reggio di Calabria,SP
Bolzano,0,93,175,503,351,201,396,528,368,408,...,564,1192,634,1451,788,1971,1466,1190,1492,5820
Verona,404,0,115,407,370,142,190,354,230,247,...,447,1001,547,1282,716,1843,1357,1084,1351,5763
Milano,681,103,0,203,605,352,356,213,255,333,...,523,1247,626,1509,833,2114,1544,1242,1462,5701
Torino,1059,197,110,0,816,529,526,222,308,426,...,572,1428,677,1679,916,2301,1676,1349,1533,5629
Trieste,674,163,299,744,0,165,405,719,400,385,...,467,852,508,1086,596,1547,1152,928,1279,5874
Venice,453,73,204,565,193,0,231,516,287,273,...,428,871,507,1143,641,1661,1234,986,1291,5811
Bologna,726,80,168,458,387,188,0,341,135,112,...,331,773,447,1070,622,1635,1210,952,1195,5738
Genoa,968,149,100,193,686,420,341,0,164,266,...,439,1142,562,1407,788,2012,1476,1174,1350,5646
Pisa,1026,147,183,409,581,356,206,250,0,92,...,287,809,427,1090,640,1680,1239,968,1146,5670
Florence,994,138,209,492,489,295,149,354,80,0,...,252,668,388,970,578,1545,1145,888,1093,5701


### Estrutura do agente:

O agente de busca neste problema é o algoritmo de busca A* que é implementado pela função ```astar_search()```, definida como:

In [8]:
start_node_value = 0
def astar_search(graph, start, end, start_node_value):
    # Create lists for open nodes and closed nodes
    open = []
    closed = []
    opened_list = []
    closed_list = []
    opened_list_entry = []
    closed_list_entry = []
    next_start_node_value = 0
    nodes_values = []
    heuristics = {}
    
    station_index = stations_list.index(end)
    heuristics = [value for value in heuristics_dict.values()][station_index]
    #print(heuristics)

    # Create a start node and an goal node
    start_node = Node(start, None)
    goal_node = Node(end, None)

    # Add the start node
    open.append(start_node)
    #To save last execution g value
    start_node.g = start_node.g + start_node_value
    # Loop until the open list is empty
    while len(open) > 0:
        opened_list_entry = []
        closed_list_entry = []

        # Sort the open list to get the node with the lowest cost first
        open.sort()

        # Get the node with the lowest cost
        current_node = open.pop(0)

        # Add the current node to the closed list
        closed.append(current_node)
        closed_list_entry.append(current_node.name)
        closed_list.append(closed_list_entry)
        
        
        # Check if we have reached the goal, return the path
        if current_node == goal_node:
            path = []
            while current_node != start_node:
                path.append(current_node.name + ': ' + str(current_node.g))
                nodes_values.append(current_node.g)
                current_node = current_node.parent
            path.append(start_node.name + ': ' + str(start_node.g))
            # Return reversed path
            next_start_node_value = nodes_values[0]
            return path[::-1], opened_list, closed_list, next_start_node_value

        # Get neighbours
        neighbors = graph.get(current_node.name)

        # Loop neighbors
        for key, value in neighbors.items():

            # Create a neighbor node
            neighbor = Node(key, current_node)

            # Check if the neighbor is in the closed list
            if(neighbor in closed):
                continue

            # Calculate full path cost

            neighbor.g = current_node.g + graph.get(current_node.name, neighbor.name)
            neighbor.h = heuristics.get(neighbor.name)
            neighbor.f = neighbor.g + neighbor.h
            # Check if neighbor is in open list and if it has a lower f value
            if(add_to_open(open, neighbor) == True):
                # Everything is green, add neighbor to open list
                open.append(neighbor)
                opened_list_entry.append(neighbor.name)
        opened_list.append(opened_list_entry)
        close_list.append(closed_list)
    # Return None, no path is found
    return None

def print_results(path, opened_list, close_list, custo_total, start, end):
    print('Caminho de '+start+' à '+ end+':')
    print(path)
    print('\n')
    print('Custo:')
    print(custo_total)
    print('\n')
    print('Nós abertos por execução:')
    print(opened_list)
    print('\n')
    print('Nós fechados por execução:')
    print(close_list)
    return

In [17]:
# Check if a neighbor should be added to open list
def add_to_open(open, neighbor):
    for node in open:
        if (neighbor == node and neighbor.f > node.f):
            return False
    return True

def gather_paths(path, new_path_piece):
    for i in new_path_piece:
        path.append(i)
    return path

Este trecho é uma adaptação do código cedido por [annytab.com](https://www.annytab.com/a-star-search-algorithm-in-python/)[1] ao domínio do problema.

## 3. Resultados

Considerando o estado inicial do problema e os possíveis estados finais, constata-se que o melhor caminho está contido entre as seguintes possibilidades:

<img src="https://guiati9.dev/img/posts_img/hipoteses.png" style="margin-left:auto; margin-right:auto;"/>


- **Hipótese 1:** SP -> ... -> Rome -> ... -> Taranto -> ... -> Verona
- **Hipótese 2:** SP -> ... -> Rome -> ... -> Verona -> ... -> Taranto
- **Hipótese 3:** SP -> ... -> Taranto -> ... -> Verona -> ... -> Rome
- **Hipótese 4:** SP -> ... -> Taranto -> ... -> Rome -> ... -> Verona
- **Hipótese 5:** SP -> ... -> Verona -> ... -> Taranto -> ... -> Rome
- **Hipótese 6:** SP -> ... -> Verona -> ... -> Rome -> ... -> Taranto

As reticências (...) indicam variações de nós a serem visitados de acordo com a tomada de decisão do algoritmo.

---

### Hipótese 1: 

*SP -> ... -> Rome -> ... -> Taranto -> ... -> Verona*

In [10]:
#SP to Rome
new_path_piece = []
open_list = []
close_list = []
start_node_value = 0
path = []
new_path_piece, opened_list, closed_list, start_node_value = astar_search(graph, 'SP', 'Rome', start_node_value)
gather_paths(path, new_path_piece)
open_list.append(opened_list)
close_list.append(closed_list)
path.pop()

#Rome to Taranto
new_path_piece = []
new_path_piece, opened_list, closed_list, start_node_value = astar_search(graph, 'Rome', 'Taranto', start_node_value)
gather_paths(path, new_path_piece)
open_list.append(opened_list)
close_list.append(closed_list)
new_path_piece = []
path.pop()

#Taranto to Verona
new_path_piece = []
new_path_piece, opened_list, closed_list, start_node_value = astar_search(graph, 'Taranto', 'Verona', start_node_value)
gather_paths(path, new_path_piece)
open_list.append(opened_list)
close_list.append(closed_list)

#results
print_results(path, open_list, closed_list, start_node_value, 'SP', 'Verona')


Caminho de SP à Verona:
['SP: 0', 'Venice: 4658', 'Bologna: 4832', 'Ancona: 5006', 'Rome: 5130', 'Pescara: 5204', 'Foggia: 5328', 'Bari: 5452', 'Taranto: 5501', 'Bari: 5550', 'Foggia: 5674', 'Pescara: 5798', 'Ancona: 5872', 'Bologna: 6046', 'Verona: 6145']


Custo:
6145


Nós abertos por execução:
[[['Rome', 'Bologna', 'Milano', 'Venice'], ['Trieste', 'Verona', 'Bologna'], ['Ancona', 'Florence', 'Genoa', 'Milano'], [], ['Rome', 'Pescara']], [['Pescara', 'Ancona', 'Florence', 'Pisa', 'SP', 'Naples'], ['Foggia'], ['Bari'], ['Taranto', 'Brindisi']], [['Naples', 'Reggio di Calabria', 'Bari'], ['Foggia', 'Brindisi'], ['Rome'], ['Pescara'], ['Ancona', 'Rome'], ['Bologna'], ['Verona', 'Venice', 'SP', 'Florence', 'Genoa', 'Milano']]]


Nós fechados por execução:
[['Taranto'], ['Bari'], ['Naples'], ['Foggia'], ['Pescara'], ['Ancona'], ['Bologna'], ['Verona']]


### Hipótese 2: 

*SP -> ... -> Rome -> ... -> Verona -> ... -> Taranto*

In [11]:
#SP to Rome
new_path_piece = []
open_list = []
close_list = []
start_node_value = 0
path = []
new_path_piece, opened_list, closed_list, start_node_value = astar_search(graph, 'SP', 'Rome', start_node_value)
gather_paths(path, new_path_piece)
open_list.append(opened_list)
close_list.append(closed_list)
path.pop()

#Rome to Verona
new_path_piece = []
new_path_piece, opened_list, closed_list, start_node_value = astar_search(graph, 'Rome', 'Verona', start_node_value)
gather_paths(path, new_path_piece)
open_list.append(opened_list)
close_list.append(closed_list)
new_path_piece = []
path.pop()

#Verona to Taranto
new_path_piece = []
new_path_piece, opened_list, closed_list, start_node_value = astar_search(graph, 'Verona', 'Taranto', start_node_value)
gather_paths(path, new_path_piece)
open_list.append(opened_list)
close_list.append(closed_list)

#results
print_results(path, open_list, close_list, start_node_value, 'SP', 'Taranto')


Caminho de SP à Taranto:
['SP: 0', 'Venice: 4658', 'Bologna: 4832', 'Ancona: 5006', 'Rome: 5130', 'Ancona: 5254', 'Bologna: 5428', 'Verona: 5527', 'Bologna: 5626', 'Ancona: 5800', 'Pescara: 5874', 'Foggia: 5998', 'Bari: 6122', 'Taranto: 6171']


Custo:
6171


Nós abertos por execução:
[[['Rome', 'Bologna', 'Milano', 'Venice'], ['Trieste', 'Verona', 'Bologna'], ['Ancona', 'Florence', 'Genoa', 'Milano'], [], ['Rome', 'Pescara']], [['Pescara', 'Ancona', 'Florence', 'Pisa', 'SP', 'Naples'], ['Bologna'], ['Verona', 'Venice', 'Genoa', 'Milano'], [], ['Foggia'], ['Genoa']], [['Milano', 'Bolzano', 'Bologna', 'Venice'], ['Ancona', 'SP', 'Florence', 'Genoa'], ['Rome', 'Pescara'], ['Foggia'], ['Bari', 'Naples'], ['Taranto', 'Brindisi']]]


Nós fechados por execução:
[[['SP'], ['Venice'], ['Bologna'], ['Trieste'], ['Ancona'], ['Rome']], [['SP'], ['Venice'], ['Bologna'], ['Trieste'], ['Ancona'], ['Rome']], [['SP'], ['Venice'], ['Bologna'], ['Trieste'], ['Ancona'], ['Rome']], [['SP'], ['Venice'], ['

### Hipótese 3: 

*SP -> ... -> Taranto -> ... -> Verona -> ... -> Rome*

In [12]:
#SP to Taranto
new_path_piece = []
open_list = []
close_list = []
start_node_value = 0
path = []
new_path_piece, opened_list, closed_list, start_node_value = astar_search(graph, 'SP', 'Taranto', start_node_value)
gather_paths(path, new_path_piece)
open_list.append(opened_list)
close_list.append(closed_list)
path.pop()

#Taranto to Verona
new_path_piece = []
new_path_piece, opened_list, closed_list, start_node_value = astar_search(graph, 'Taranto', 'Verona', start_node_value)
gather_paths(path, new_path_piece)
open_list.append(opened_list)
close_list.append(closed_list)
new_path_piece = []
path.pop()

#Verona to Rome
new_path_piece = []
new_path_piece, opened_list, closed_list, start_node_value = astar_search(graph, 'Verona', 'Rome', start_node_value)
gather_paths(path, new_path_piece)
open_list.append(opened_list)
close_list.append(closed_list)

#results
print_results(path, open_list, close_list, start_node_value, 'SP', 'Rome')


Caminho de SP à Rome:
['SP: 0', 'Venice: 4658', 'Bologna: 4832', 'Ancona: 5006', 'Pescara: 5080', 'Foggia: 5204', 'Bari: 5328', 'Taranto: 5377', 'Bari: 5426', 'Foggia: 5550', 'Pescara: 5674', 'Ancona: 5748', 'Bologna: 5922', 'Verona: 6021', 'Bologna: 6120', 'Ancona: 6294', 'Rome: 6418']


Custo:
6418


Nós abertos por execução:
[[['Rome', 'Bologna', 'Milano', 'Venice'], ['Trieste', 'Verona', 'Bologna'], [], ['Ancona', 'Florence', 'Genoa', 'Milano'], ['Rome', 'Pescara'], ['Foggia'], ['Bari', 'Naples'], ['Taranto', 'Brindisi']], [['Naples', 'Reggio di Calabria', 'Bari'], ['Foggia', 'Brindisi'], ['Rome'], ['Pescara'], ['Ancona', 'Rome'], ['Bologna'], ['Verona', 'Venice', 'SP', 'Florence', 'Genoa', 'Milano']], [['Milano', 'Bolzano', 'Bologna', 'Venice'], ['Ancona', 'SP', 'Florence', 'Genoa'], ['Rome', 'Pescara']]]


Nós fechados por execução:
[[['SP'], ['Venice'], ['Trieste'], ['Bologna'], ['Ancona'], ['Pescara'], ['Foggia'], ['Bari'], ['Taranto']], [['SP'], ['Venice'], ['Trieste'], ['Bolo

### Hipótese 4: 

*SP -> ... -> Taranto -> ... -> Rome -> ... -> Verona*

In [13]:
#SP to Taranto
new_path_piece = []
open_list = []
close_list = []
start_node_value = 0
path = []
new_path_piece, opened_list, closed_list, start_node_value = astar_search(graph, 'SP', 'Taranto', start_node_value)
gather_paths(path, new_path_piece)
open_list.append(opened_list)
close_list.append(closed_list)
path.pop()

#Taranto to Rome
new_path_piece = []
new_path_piece, opened_list, closed_list, start_node_value = astar_search(graph, 'Taranto', 'Rome', start_node_value)
gather_paths(path, new_path_piece)
open_list.append(opened_list)
close_list.append(closed_list)
new_path_piece = []
path.pop()

#Rome to Verona
new_path_piece = []
new_path_piece, opened_list, closed_list, start_node_value = astar_search(graph, 'Rome', 'Verona', start_node_value)
gather_paths(path, new_path_piece)
open_list.append(opened_list)
close_list.append(closed_list)

#results
print_results(path, open_list, close_list, start_node_value, 'SP', 'Verona')


Caminho de SP à Verona:
['SP: 0', 'Venice: 4658', 'Bologna: 4832', 'Ancona: 5006', 'Pescara: 5080', 'Foggia: 5204', 'Bari: 5328', 'Taranto: 5377', 'Naples: 5526', 'Rome: 5774', 'Ancona: 5898', 'Bologna: 6072', 'Verona: 6171']


Custo:
6171


Nós abertos por execução:
[[['Rome', 'Bologna', 'Milano', 'Venice'], ['Trieste', 'Verona', 'Bologna'], [], ['Ancona', 'Florence', 'Genoa', 'Milano'], ['Rome', 'Pescara'], ['Foggia'], ['Bari', 'Naples'], ['Taranto', 'Brindisi']], [['Naples', 'Reggio di Calabria', 'Bari'], ['Foggia', 'Rome']], [['Pescara', 'Ancona', 'Florence', 'Pisa', 'SP', 'Naples'], ['Bologna'], ['Verona', 'Venice', 'Genoa', 'Milano'], [], ['Foggia'], ['Genoa']]]


Nós fechados por execução:
[[['SP'], ['Venice'], ['Trieste'], ['Bologna'], ['Ancona'], ['Pescara'], ['Foggia'], ['Bari'], ['Taranto']], [['SP'], ['Venice'], ['Trieste'], ['Bologna'], ['Ancona'], ['Pescara'], ['Foggia'], ['Bari'], ['Taranto']], [['SP'], ['Venice'], ['Trieste'], ['Bologna'], ['Ancona'], ['Pescara'], ['Fog

### Hipótese 5: 

*SP -> ... -> Verona -> ... -> Taranto -> ... -> Rome*

In [14]:
#SP to Verona
new_path_piece = []
open_list = []
close_list = []
start_node_value = 0
path = []
new_path_piece, opened_list, closed_list, start_node_value = astar_search(graph, 'SP', 'Verona', start_node_value)
gather_paths(path, new_path_piece)
open_list.append(opened_list)
close_list.append(closed_list)
path.pop()

#Verona to Taranto
new_path_piece = []
new_path_piece, opened_list, closed_list, start_node_value = astar_search(graph, 'Verona', 'Taranto', start_node_value)
gather_paths(path, new_path_piece)
open_list.append(opened_list)
close_list.append(closed_list)
new_path_piece = []
path.pop()

#Taranto to Rome
new_path_piece = []
new_path_piece, opened_list, closed_list, start_node_value = astar_search(graph, 'Taranto', 'Rome', start_node_value)
gather_paths(path, new_path_piece)
open_list.append(opened_list)
close_list.append(closed_list)

#results
print_results(path, open_list, close_list, start_node_value, 'SP', 'Rome')


Caminho de SP à Rome:
['SP: 0', 'Venice: 4658', 'Verona: 4782', 'Bologna: 4881', 'Ancona: 5055', 'Pescara: 5129', 'Foggia: 5253', 'Bari: 5377', 'Taranto: 5426', 'Naples: 5575', 'Rome: 5823']


Custo:
5823


Nós abertos por execução:
[[['Rome', 'Bologna', 'Milano', 'Venice'], ['Trieste', 'Verona', 'Bologna']], [['Milano', 'Bolzano', 'Bologna', 'Venice'], ['Ancona', 'SP', 'Florence', 'Genoa'], ['Rome', 'Pescara'], ['Foggia'], ['Bari', 'Naples'], ['Taranto', 'Brindisi']], [['Naples', 'Reggio di Calabria', 'Bari'], ['Foggia', 'Rome']]]


Nós fechados por execução:
[[['SP'], ['Venice'], ['Verona']], [['SP'], ['Venice'], ['Verona']], [['SP'], ['Venice'], ['Verona']], [['Verona'], ['Bologna'], ['Ancona'], ['Pescara'], ['Foggia'], ['Bari'], ['Taranto']], [['Verona'], ['Bologna'], ['Ancona'], ['Pescara'], ['Foggia'], ['Bari'], ['Taranto']], [['Verona'], ['Bologna'], ['Ancona'], ['Pescara'], ['Foggia'], ['Bari'], ['Taranto']], [['Verona'], ['Bologna'], ['Ancona'], ['Pescara'], ['Foggia'], ['Bari

### Hipótese 6: 

*SP -> ... -> Verona -> ... -> Rome -> ... -> Taranto*

In [15]:
#SP to Verona
new_path_piece = []
open_list = []
close_list = []
start_node_value = 0
path = []
new_path_piece, opened_list, closed_list, start_node_value = astar_search(graph, 'SP', 'Verona', start_node_value)
gather_paths(path, new_path_piece)
open_list.append(opened_list)
close_list.append(closed_list)
path.pop()

#Verona to Rome
new_path_piece = []
new_path_piece, opened_list, closed_list, start_node_value = astar_search(graph, 'Verona', 'Rome', start_node_value)
gather_paths(path, new_path_piece)
open_list.append(opened_list)
close_list.append(closed_list)
new_path_piece = []
path.pop()

#Rome to Taranto
new_path_piece = []
new_path_piece, opened_list, closed_list, start_node_value = astar_search(graph, 'Rome', 'Taranto', start_node_value)
gather_paths(path, new_path_piece)
open_list.append(opened_list)
close_list.append(closed_list)

#results
print_results(path, open_list, close_list, start_node_value, 'SP', 'Taranto')


Caminho de SP à Taranto:
['SP: 0', 'Venice: 4658', 'Verona: 4782', 'Bologna: 4881', 'Ancona: 5055', 'Rome: 5179', 'Pescara: 5253', 'Foggia: 5377', 'Bari: 5501', 'Taranto: 5550']


Custo:
5550


Nós abertos por execução:
[[['Rome', 'Bologna', 'Milano', 'Venice'], ['Trieste', 'Verona', 'Bologna']], [['Milano', 'Bolzano', 'Bologna', 'Venice'], ['Ancona', 'SP', 'Florence', 'Genoa'], ['Rome', 'Pescara']], [['Pescara', 'Ancona', 'Florence', 'Pisa', 'SP', 'Naples'], ['Foggia'], ['Bari'], ['Taranto', 'Brindisi']]]


Nós fechados por execução:
[[['SP'], ['Venice'], ['Verona']], [['SP'], ['Venice'], ['Verona']], [['SP'], ['Venice'], ['Verona']], [['Verona'], ['Bologna'], ['Ancona'], ['Rome']], [['Verona'], ['Bologna'], ['Ancona'], ['Rome']], [['Verona'], ['Bologna'], ['Ancona'], ['Rome']], [['Verona'], ['Bologna'], ['Ancona'], ['Rome']], [['Rome'], ['Pescara'], ['Foggia'], ['Bari'], ['Taranto']], [['Rome'], ['Pescara'], ['Foggia'], ['Bari'], ['Taranto']], [['Rome'], ['Pescara'], ['Foggia'], ['Ba

A **Hipótese 6** foi a que apresentou o menor custo, portanto podemos considerá-la a solução do problema estabelecido pelo projeto. 

## Comparação com heurística não admissível

Uma heurística não admissível é aquela que superestima o valor do real custo até o nó, ou seja, apresenta um valor superior ao custo até o nó. É necessário lembrar que o Algoritmo A* sempre busca a rota com menor custo total, dessa forma, quando aplicado uma heurística não admissível, os aspectos considerados na construção da heurística passam a ser mais importantes que o custo real do caminho proposto pela busca, tornando seu resultado impreciso.

Como dito anteriormente, todos os valores obtidos através do produto entre a distância em linha reta da estação ao destino e o preço médio de hoteis na estação dividido por 100 compõe uma heurística admissível justamente pelo fato de que a divisão, igualmente aplicada à todos os itens, reduz o impacto da função heurística na execução do algoritmo, mas que ainda apresentam valores úteis para o processo de tomada de decisão do algoritmo.

Sendo assim, é possível citar como exemplo de uma heurística não admissível a mesma fórmula aplicada anteriormente, porém, sem a divisão dos valores por 100 (h(n) = d * p). Segue exemplo de aplicação de heurística admissível à hipótese ótima (*SP -> ... -> Verona -> ... -> Rome -> ... -> Taranto*):

In [16]:
###### Aplicando heurística não admissível (distância em linha reta ao destino * preco médio de hotel na região)
dict_hoteis = precos_hotel.to_dict()
for station in stations_list:
    teste = tabela_heuristicas[station].to_dict()
    #print(station)
    #print(teste)
    for key in dict_hoteis:
        teste[key] = teste.get(key, 1) * dict_hoteis[station][0]
    for key in teste:
        tabela_heuristicas[station][key] = teste[key]
heuristics_dict = tabela_heuristicas.to_dict()

#SP to Verona
open_list = []
close_list = []
start_node_value = 0
path = []
new_path_piece, opened_list, closed_list, start_node_value = astar_search(graph, 'SP', 'Verona', start_node_value)
gather_paths(path, new_path_piece)
open_list.append(opened_list)
close_list.append(closed_list)
path.pop()

#Verona to Rome
new_path_piece = []
new_path_piece, opened_list, closed_list, start_node_value = astar_search(graph, 'Verona', 'Rome', start_node_value)
gather_paths(path, new_path_piece)
open_list.append(opened_list)
close_list.append(closed_list)
new_path_piece = []
path.pop()

#Rome to Taranto
new_path_piece = []
new_path_piece, opened_list, closed_list, start_node_value = astar_search(graph, 'Rome', 'Taranto', start_node_value)
gather_paths(path, new_path_piece)
open_list.append(opened_list)
close_list.append(closed_list)

#results
print_results(path, open_list, close_list, start_node_value, 'SP', 'Taranto')

Caminho de SP à Taranto:
['SP: 0', 'Venice: 4658', 'Verona: 4782', 'Bologna: 4881', 'SP: 11191', 'Rome: 16751', 'Naples: 16999', 'Taranto: 17148']


Custo:
17148


Nós abertos por execução:
[[['Rome', 'Bologna', 'Milano', 'Venice'], ['Trieste', 'Verona', 'Bologna']], [['Milano', 'Bolzano', 'Bologna', 'Venice'], ['Ancona', 'SP', 'Florence', 'Genoa'], ['Rome']], [['Pescara', 'Ancona', 'Florence', 'Pisa', 'SP', 'Naples'], ['Bologna', 'Milano', 'Venice'], ['Foggia', 'Reggio di Calabria', 'Taranto']]]


Nós fechados por execução:
[[['SP'], ['Venice'], ['Verona']], [['SP'], ['Venice'], ['Verona']], [['SP'], ['Venice'], ['Verona']], [['Verona'], ['Bologna'], ['SP'], ['Rome']], [['Verona'], ['Bologna'], ['SP'], ['Rome']], [['Verona'], ['Bologna'], ['SP'], ['Rome']], [['Verona'], ['Bologna'], ['SP'], ['Rome']], [['Rome'], ['SP'], ['Naples'], ['Taranto']], [['Rome'], ['SP'], ['Naples'], ['Taranto']], [['Rome'], ['SP'], ['Naples'], ['Taranto']], [['Rome'], ['SP'], ['Naples'], ['Taranto']]]


O custo apresentado para o caminho, já comprovado como ideal, pelo algoritmo aplicado à uma heurística não admissível é 17148. 

## Conclusão

O algoritmo A*, apesar de apresentar um custo de tempo e espaço maior que outros algoritmos de busca convencionais, sempre será eficaz para apresentar o caminho de menor custo. Para isso, deve contar com o auxílio de uma função heurística admissível de qualidade. 

É possível destacar a importância da heurística e sua admissibilidade na aplicação do algoritmo uma vez que o resultado apresentado ao ser aplicado  em uma heurística não admissível, mesmo encontrando um caminho, foi insatisfatório devido ao custo ser maior do que deveria.

Este projeto apresentou a rota de São Paulo para 3 destinos da Itália, porém, entre os resultados do seu desenvolvimento, está um programa de fácil manuseio que permite a aplicação do algoritmo partindo de/com destino de/à diversos destinos.

Outro ponto que vale ser ressaltado é a importância da qualidade dos dados utilizados. Contar com uma boa função heurística é essencial, porém ineficaz se aplicada à dados ruidosos e não tratados.

## Referências

[1] https://www.annytab.com/a-star-search-algorithm-in-python/ - Acessado em 29 de maio de 2020.